In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2 as cv
import os

# Set various TF options
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# Set location of CUDA installation
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/opt/cuda'

import tensorflow as tf

tf.debugging.set_log_device_placement(False)

print("TF version:", tf.__version__)
print("TF device:", tf.config.list_physical_devices('GPU'))

# Note: Disable warning about NUMA node 
# $ for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done

In [ ]:
from common import CV_DATASETS_DIR
from pathlib import Path

dataset_path = Path(CV_DATASETS_DIR, 'homer_bart_2.zip')  
archive = tf.keras.utils.get_file(origin=dataset_path.as_uri(), extract=True)
data_dir = Path(archive).with_suffix('')

In [19]:
image_shape = (128, 128, 3)

train_ds = tf.keras.utils.image_dataset_from_directory(
    Path(data_dir, 'training_set'),
    image_size=image_shape[:2],
    batch_size=8,
    labels='inferred'
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    Path(data_dir, 'test_set'),
    image_size=image_shape[:2],
    batch_size=1,
    labels='inferred',
    shuffle=False
)

Found 215 files belonging to 2 classes.
Found 54 files belonging to 2 classes.


In [20]:
hidden_layer_neurons = int((1152 + 2) / 2)

model = tf.keras.Sequential()

model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.RandomRotation(0.5))
model.add(tf.keras.layers.RandomFlip(mode="horizontal_and_vertical"))
model.add(tf.keras.layers.RandomZoom(height_factor=0.3, width_factor=0.3))

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=hidden_layer_neurons, activation='relu'))
model.add(tf.keras.layers.Dense(units=hidden_layer_neurons, activation='relu'))
model.add(tf.keras.layers.Dense(units=2, activation='softmax'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_2 (Rescaling)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_2               │ ?                      │   0 (unbuilt) │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip_2 (RandomFlip)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_2 (RandomZoom)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=30
)

Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6100 - loss: 0.7637 - val_accuracy: 0.5185 - val_loss: 0.8106
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6197 - loss: 0.6172 - val_accuracy: 0.5370 - val_loss: 0.6864
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6896 - loss: 0.5822 - val_accuracy: 0.7222 - val_loss: 0.6017
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7116 - loss: 0.5685 - val_accuracy: 0.7593 - val_loss: 0.5801
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6747 - loss: 0.5279 - val_accuracy: 0.5741 - val_loss: 0.8187
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7813 - loss: 0.4504 - val_accuracy: 0.7778 - val_loss: 0.6865
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7907 - loss: 0.4818 - val_accuracy: 0.7593 - val_loss: 0.5059
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8503 - loss: 0.3760 - val_accuracy: 0.5556 - val_loss

In [22]:
predictions = model.predict(val_ds)
predictions = np.argmax(predictions, axis=1)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step


In [23]:
classes = []
for image, clazz in val_ds.as_numpy_iterator():
    classes.append(clazz)
classes = np.asarray(classes)

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(classes, predictions)

0.7962962962962963

In [25]:
from sklearn.metrics import classification_report
print(classification_report(classes, predictions))

              precision    recall  f1-score   support

           0       0.77      0.86      0.81        28
           1       0.83      0.73      0.78        26

    accuracy                           0.80        54
   macro avg       0.80      0.79      0.79        54
weighted avg       0.80      0.80      0.80        54

